In [74]:
rng = np.random.default_rng(seed=12345)

# PHỤ LỤC A: KỸ THUẬT NUMPY NÂNG CAO

Trong phần phụ lục này, tôi sẽ đi sâu hơn vào thư viện NumPy dành cho việc tính toán mảng. Nội dung sẽ bao gồm các chi tiết nội bộ về kiểu dữ liệu ndarray và các thao tác cũng như thuật toán nâng cao với mảng.

Phụ lục này chứa các chủ đề đa dạng và không nhất thiết phải đọc theo thứ tự. Xuyên suốt các chương, tôi sẽ tạo dữ liệu ngẫu nhiên cho nhiều ví dụ, sử dụng bộ tạo số ngẫu nhiên mặc định trong mô-đun numpy.random.

In [ ]:
# Importing các thư viện Python cần thiết
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

In [ ]:
rng = np.random.default_rng(seed=12345)

# A.1 Cấu trúc bên trong của đối tượng ndarray


### Các thành phần của ndarray


Thư viện NumPy cung cấp kiểu dữ liệu ndarray cho phép diễn giải một khối dữ liệu có kiểu đồng nhất (liên tục hoặc có bước nhảy) dưới dạng một đối tượng mảng nhiều chiều. Kiểu dữ liệu, hay dtype, quyết định cách dữ liệu được hiểu là số thực, số nguyên, giá trị Boolean, hoặc các kiểu khác mà chúng ta đã tìm hiểu.

Một phần khiến ndarray trở nên linh hoạt là vì mỗi đối tượng mảng là một "view" có bước nhảy (strided view) trên một khối dữ liệu. Bạn có thể thắc mắc, ví dụ, tại sao cách truy cập mảng arr[::2, ::-1] lại không sao chép dữ liệu. Lý do là vì ndarray không chỉ là một khối bộ nhớ và kiểu dữ liệu; nó còn chứa thông tin về bước nhảy, cho phép mảng di chuyển qua bộ nhớ với các kích thước bước khác nhau.

Cụ thể hơn, bên trong ndarray bao gồm các thành phần sau:

Một con trỏ tới dữ liệu — tức là một khối dữ liệu nằm trong RAM hoặc trong một tệp ánh xạ bộ nhớ

Kiểu dữ liệu (dtype) mô tả các ô giá trị có kích thước cố định trong mảng

Một tuple biểu thị hình dạng (shape) của mảng

Một tuple các bước nhảy (strides) — các số nguyên biểu thị số byte cần “bước” để di chuyển một phần tử theo từng chiều

In [ ]:
np.ones((10, 5)).shape

(10, 5)

In [ ]:
np.ones((3, 4, 5), dtype=np.float64).strides

(160, 40, 8)

### Hệ thống phân cấp kiểu dữ liệu (dtype)


In [ ]:
ints = np.ones(10, dtype=np.uint16)
floats = np.ones(10, dtype=np.float32)
np.issubdtype(ints.dtype, np.integer)

True

In [ ]:
np.issubdtype(floats.dtype, np.floating)

True

In [ ]:
np.float64.mro()

[numpy.float64,
 numpy.floating,
 numpy.inexact,
 numpy.number,
 numpy.generic,
 float,
 object]

In [ ]:
np.issubdtype(ints.dtype, np.number)

True

#A.2. Thao tác mảng nâng cao

Có nhiều cách để làm việc với mảng ngoài các phương pháp như fancy indexing (lập chỉ mục nâng cao), slicing (cắt lát), và Boolean subsetting (lọc theo điều kiện logic). Mặc dù phần lớn các tác vụ nặng trong ứng dụng phân tích dữ liệu đã được xử lý bởi các hàm cấp cao trong pandas, nhưng có thể đến một lúc nào đó bạn sẽ cần viết một thuật toán dữ liệu mà không có sẵn trong các thư viện hiện có.

## Định hình lại mảng

Trong nhiều trường hợp, một mảng có thể được chuyển đổi từ hình dạng này sang hình dạng khác mà không cần sao chép bất kỳ dữ liệu nào. Để làm điều này, hãy truyền một tuple chỉ định hình dạng mới vào phương thức reshape của mảng. Ví dụ, giả sử có một mảng giá trị một chiều và cần sắp xếp lại nó thành một ma trận.

In [75]:
import numpy as np
arr = np.arange(8)

In [76]:
arr

array([0, 1, 2, 3, 4, 5, 6, 7])

In [77]:
arr.reshape((4, 2))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7]])

Một mảng đa chiều cũng có thể được định hình lại:

In [78]:
arr.reshape((4, 2)).reshape((2, 4))

array([[0, 1, 2, 3],
       [4, 5, 6, 7]])

Một trong các chiều của shape (hình dạng) được truyền vào có thể là –1, khi đó giá trị của chiều đó sẽ được tự động suy ra từ dữ liệu:

In [79]:
arr = np.arange(15)

In [80]:
arr.reshape((5, -1))

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

Vì thuộc tính shape của một mảng là một tuple, nên nó cũng có thể được truyền vào hàm reshape:


In [81]:
other_arr = np.ones((3, 5))

In [82]:
other_arr.shape

(3, 5)

In [83]:
arr.reshape(other_arr.shape)

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

Thao tác ngược lại của reshape (chuyển từ nhiều chiều về một chiều) thường được gọi là "làm phẳng" (flattening) hoặc "duỗi thẳng" (raveling):

In [84]:
arr = np.arange(15).reshape((5, 3))

In [85]:
arr

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [86]:
arr.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

Hàm ravel không tạo ra bản sao của dữ liệu gốc nếu các giá trị trong kết quả vốn đã liền kề trong mảng ban đầu.

Phương thức flatten hoạt động tương tự như ravel, ngoại trừ việc nó luôn luôn trả về một bản sao của dữ liệu.

In [87]:
arr.flatten()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

Dữ liệu có thể được reshape (định hình lại) hoặc ravel (làm phẳng) theo các thứ tự khác nhau. Đây là một chủ đề hơi phức tạp đối với người mới dùng NumPy, vì vậy nó sẽ được trình bày ở tiểu mục tiếp theo.

## C Versus FORTRAN Order

NumPy có thể thích ứng với nhiều cách bố trí dữ liệu khác nhau trong bộ nhớ. Theo mặc định, các mảng NumPy được tạo theo thứ tự row-major (ưu tiên hàng). Về mặt không gian, điều này có nghĩa là nếu bạn có một mảng hai chiều, các phần tử trong mỗi hàng sẽ được lưu trữ ở các vị trí bộ nhớ liền kề nhau. Lựa chọn thay thế là thứ tự column-major (ưu tiên cột), có nghĩa là các giá trị trong mỗi cột sẽ được lưu trữ ở các vị trí bộ nhớ liền kề nhau.

Vì lý do lịch sử, thứ tự row-major (ưu tiên hàng) và column-major (ưu tiên cột) còn được gọi tương ứng là thứ tự C và FORTRAN. Trong ngôn ngữ FORTRAN 77, các ma trận đều là column-major.


Các hàm như reshape và ravel chấp nhận một tham số order để chỉ định thứ tự sử dụng dữ liệu trong mảng. Tham số này thường được đặt là 'C' hoặc 'F' trong hầu hết các trường hợp

In [88]:
arr = np.arange(12).reshape((3, 4))

In [89]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [90]:
arr.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [91]:
arr.ravel('F')

array([ 0,  4,  8,  1,  5,  9,  2,  6, 10,  3,  7, 11])

Sự khác biệt cốt lõi giữa thứ tự C và FORTRAN khi reshape mảng nhiều chiều là thứ tự duyệt qua các trục (axis):

* Thứ tự C (row-major): Ưu tiên duyệt các trục bên trong trước. Nó sẽ đi hết các phần tử của hàng (trục 1) rồi mới chuyển sang hàng tiếp theo (trục 0).

* Thứ tự FORTRAN (column-major): Ưu tiên duyệt các trục bên ngoài trước. Nó sẽ lấy phần tử đầu tiên của tất cả các hàng (trục 0) rồi mới sang phần tử thứ hai.

## Ghép nối và Chia tách Mảng

Hàm numpy.concatenate nhận vào một chuỗi (ví dụ: tuple, list) các mảng và ghép nối chúng lại theo thứ tự dọc theo một trục (axis) đã cho:

In [92]:
arr1 = np.array([[1, 2, 3], [4, 5, 6]])

In [93]:
arr2 = np.array([[7, 8, 9], [10, 11, 12]])

In [94]:
np.concatenate([arr1, arr2], axis=0)

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [95]:
np.concatenate([arr1, arr2], axis=1)

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

Có một số hàm tiện ích, như vstack và hstack, dành cho các loại ghép nối phổ biến. Các thao tác trước đó có thể được biểu diễn như sau:

In [96]:
np.vstack((arr1, arr2))

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [97]:
np.hstack((arr1, arr2))

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

Ngược lại, hàm split chia một mảng thành nhiều mảng dọc theo một trục:

In [98]:
arr = rng.standard_normal((5, 2))

In [99]:
arr

array([[-1.42382504,  1.26372846],
       [-0.87066174, -0.25917323],
       [-0.07534331, -0.74088465],
       [-1.3677927 ,  0.6488928 ],
       [ 0.36105811, -1.95286306]])

In [100]:
first, second, third = np.split(arr, [1, 3])

In [101]:
first

array([[-1.42382504,  1.26372846]])

In [102]:
second

array([[-0.87066174, -0.25917323],
       [-0.07534331, -0.74088465]])

In [103]:
third

array([[-1.3677927 ,  0.6488928 ],
       [ 0.36105811, -1.95286306]])

Giá trị [1, 3] được truyền vào np.split chỉ định các chỉ số mà tại đó mảng sẽ được chia thành nhiều mảnh.

**Các hàm hỗ trợ xếp chồng: r_ và c_**

Có hai đối tượng đặc biệt trong không gian tên NumPy là r_ và c_, giúp cho việc xếp chồng các mảng trở nên ngắn gọn hơn:

In [104]:
arr = np.arange(6)

In [105]:
arr1 = arr.reshape((3, 2))

In [106]:
arr2 = rng.standard_normal((3, 2))

In [107]:
np.r_[arr1, arr2]

array([[ 0.        ,  1.        ],
       [ 2.        ,  3.        ],
       [ 4.        ,  5.        ],
       [ 2.34740965,  0.96849691],
       [-0.75938718,  0.90219827],
       [-0.46695317, -0.06068952]])

In [108]:
np.c_[np.r_[arr1, arr2], arr]

array([[ 0.        ,  1.        ,  0.        ],
       [ 2.        ,  3.        ,  1.        ],
       [ 4.        ,  5.        ,  2.        ],
       [ 2.34740965,  0.96849691,  3.        ],
       [-0.75938718,  0.90219827,  4.        ],
       [-0.46695317, -0.06068952,  5.        ]])

Ngoài ra, chúng còn có thể chuyển đổi các lát cắt (slices) thành mảng:

In [109]:
np.c_[1:6, -10:-5]

array([[  1, -10],
       [  2,  -9],
       [  3,  -8],
       [  4,  -7],
       [  5,  -6]])

Tham khảo tài liệu hướng dẫn (docstring) để biết thêm về các chức năng của c_ và r_.

## Lặp lại các Phần tử: tile và repeat

Hai công cụ hữu ích để lặp lại hoặc sao chép các mảng nhằm tạo ra các mảng lớn hơn là hàm repeat và tile. Hàm repeat lặp lại mỗi phần tử trong một mảng một số lần nhất định, tạo ra một mảng lớn hơn:

In [110]:
arr = np.arange(3)

In [111]:
arr

array([0, 1, 2])

In [112]:
arr.repeat(3)

array([0, 0, 0, 1, 1, 1, 2, 2, 2])

Theo mặc định, nếu bạn truyền vào một số nguyên, mỗi phần tử sẽ được lặp lại đúng số lần đó.

Nếu bạn truyền vào một mảng các số nguyên, mỗi phần tử có thể được lặp lại một số lần khác nhau:

In [113]:
arr.repeat([2, 3, 4])

array([0, 0, 1, 1, 1, 2, 2, 2, 2])

Đối với mảng đa chiều, các phần tử của chúng có thể được lặp lại dọc theo một trục cụ thể:

In [114]:
arr = rng.standard_normal((2, 2))

In [115]:
arr

array([[ 0.78884434, -1.25666813],
       [ 0.57585751,  1.39897899]])

In [116]:
arr.repeat(2, axis=0)

array([[ 0.78884434, -1.25666813],
       [ 0.78884434, -1.25666813],
       [ 0.57585751,  1.39897899],
       [ 0.57585751,  1.39897899]])

Lưu ý rằng nếu không có trục (axis) nào được truyền vào, mảng sẽ được làm phẳng trước, và đây có thể không phải là điều mong muốn. Tương tự, có thể truyền vào một mảng các số nguyên khi lặp lại một mảng đa chiều để lặp lại một lát cắt (slice) nhất định một số lần khác nhau:

In [117]:
arr.repeat([2, 3], axis=0)

array([[ 0.78884434, -1.25666813],
       [ 0.78884434, -1.25666813],
       [ 0.57585751,  1.39897899],
       [ 0.57585751,  1.39897899],
       [ 0.57585751,  1.39897899]])

In [118]:
arr.repeat([2, 3], axis=1)

array([[ 0.78884434,  0.78884434, -1.25666813, -1.25666813, -1.25666813],
       [ 0.57585751,  0.57585751,  1.39897899,  1.39897899,  1.39897899]])

Mặt khác, hàm tile là một cách viết tắt cho việc xếp chồng các bản sao của một mảng dọc theo một trục.

Về mặt trực quan, có thể hình dung nó tương tự như việc 'lát gạch':

In [119]:
arr

array([[ 0.78884434, -1.25666813],
       [ 0.57585751,  1.39897899]])

In [120]:
np.tile(arr, 2)

array([[ 0.78884434, -1.25666813,  0.78884434, -1.25666813],
       [ 0.57585751,  1.39897899,  0.57585751,  1.39897899]])

Tham số thứ hai là số lần lát gạch; với một số vô hướng, việc lát gạch được thực hiện theo hàng, chứ không phải theo cột. Tham số thứ hai của hàm tile cũng có thể là một tuple để chỉ định cách bố trí của việc 'lát gạch':

In [121]:
arr

array([[ 0.78884434, -1.25666813],
       [ 0.57585751,  1.39897899]])

In [122]:
np.tile(arr, (2, 1))

array([[ 0.78884434, -1.25666813],
       [ 0.57585751,  1.39897899],
       [ 0.78884434, -1.25666813],
       [ 0.57585751,  1.39897899]])

In [123]:
np.tile(arr, (3, 2))

array([[ 0.78884434, -1.25666813,  0.78884434, -1.25666813],
       [ 0.57585751,  1.39897899,  0.57585751,  1.39897899],
       [ 0.78884434, -1.25666813,  0.78884434, -1.25666813],
       [ 0.57585751,  1.39897899,  0.57585751,  1.39897899],
       [ 0.78884434, -1.25666813,  0.78884434, -1.25666813],
       [ 0.57585751,  1.39897899,  0.57585751,  1.39897899]])

## Các hàm tương đương với Lập chỉ mục Nâng cao (Fancy Indexing): take và put

Một cách để lấy và thiết lập các tập con của mảng là sử dụng "lập chỉ mục nâng cao" (fancy indexing) bằng các mảng số nguyên

In [124]:
arr = np.arange(10) * 100

In [125]:
inds = [7, 1, 2, 6]

In [126]:
arr[inds]

array([700, 100, 200, 600])

Có các phương thức ndarray khác hữu ích trong trường hợp đặc biệt khi chỉ cần lựa chọn trên một trục duy nhất:

In [127]:
arr.take(inds)

array([700, 100, 200, 600])

In [128]:
arr.put(inds, 42)

In [129]:
arr

array([  0,  42,  42, 300, 400, 500,  42,  42, 800, 900])

In [130]:
arr.put(inds, [40, 41, 42, 43])

In [131]:
arr

array([  0,  41,  42, 300, 400, 500,  43,  40, 800, 900])

Để dùng take trên các trục khác, bạn có thể truyền vào từ khóa axis:

In [132]:
inds = [2, 0, 2, 1]

In [133]:
arr = rng.standard_normal((2, 4))

In [134]:
arr

array([[ 1.32229806, -0.29969852,  0.90291934, -1.62158273],
       [-0.15818926,  0.44948393, -1.34360107, -0.08168759]])

In [135]:
arr.take(inds, axis=1)

array([[ 0.90291934,  1.32229806,  0.90291934, -0.29969852],
       [-1.34360107, -0.15818926, -1.34360107,  0.44948393]])

Hàm put không chấp nhận tham số axis, thay vào đó, nó lập chỉ mục trên phiên bản đã được làm phẳng (một chiều, theo thứ tự C) của mảng. Do đó, khi cần thiết lập các phần tử bằng một mảng chỉ số trên các trục khác, tốt nhất là sử dụng cách lập chỉ mục bằng dấu ngoặc vuông []

#**A.3. Broadcasting (Quy tắc Lan truyền)**

Broadcasting quy định cách các phép toán hoạt động giữa những mảng có hình dạng (shape) khác nhau. Đây là một tính năng mạnh mẽ nhưng có thể gây bối rối, ngay cả với những người dùng có kinh nghiệm. Ví dụ đơn giản nhất về broadcasting xảy ra khi kết hợp một giá trị vô hướng với một mảng:

In [136]:
arr = np.arange(5)

In [137]:
arr

array([0, 1, 2, 3, 4])

In [138]:
arr * 4

array([ 0,  4,  8, 12, 16])

Ở đây, ta nói rằng giá trị vô hướng 4 đã được "lan truyền" (broadcast) tới tất cả các phần tử khác trong phép nhân.

Ví dụ, ta có thể "khử trung bình" (demean) mỗi cột của một mảng bằng cách trừ đi giá trị trung bình của các cột đó. Trong trường hợp này, chỉ cần trừ đi một mảng chứa giá trị trung bình của mỗi cột là đủ:

In [139]:
arr = rng.standard_normal((4, 3))

In [140]:
arr.mean(0)

array([0.1205802 , 0.24301074, 0.14436756])

In [141]:
demeaned = arr - arr.mean(0)

In [142]:
demeaned

array([[ 1.60415973,  2.37514869,  0.63299379],
       [ 0.708053  , -1.20199905, -1.35375584],
       [-1.53287221,  0.29853609,  0.60757184],
       [-0.77934052, -1.47168573,  0.11319021]])

In [144]:
demeaned.mean(0)

array([ 5.55111512e-17, -1.11022302e-16,  0.00000000e+00])

Thực hiện phép toán giữa các mảng có hình dạng không tương thích (ví dụ: một mảng 2 chiều với một mảng 1 chiều chứa giá trị trung bình của các hàng) được gọi là broadcasting. Thao tác này hoàn toàn có thể thực hiện được nhưng phải tuân theo những quy tắc cụ thể để đảm bảo tính toán đúng.

Hãy xem xét ví dụ trước: giả sử thay vì trừ trung bình cột, chúng ta muốn trừ đi giá trị trung bình của mỗi hàng. Vì arr.mean(0) (trung bình cột) có độ dài 3, nó tương thích để lan truyền trên axis 0 vì chiều cuối cùng của arr cũng là 3.

Tuy nhiên, theo quy tắc, để thực hiện phép trừ trên axis 1 (tức là trừ đi giá trị trung bình của hàng), mảng nhỏ hơn phải có hình dạng là (4, 1):

In [145]:
arr

array([[ 1.72473993,  2.61815943,  0.77736134],
       [ 0.8286332 , -0.95898831, -1.20938829],
       [-1.41229201,  0.54154683,  0.7519394 ],
       [-0.65876032, -1.22867499,  0.25755777]])

In [146]:
row_means = arr.mean(1)

In [147]:
row_means.shape

(4,)

In [148]:
row_means.reshape((4, 1))

array([[ 1.70675357],
       [-0.44658113],
       [-0.03960193],
       [-0.54329251]])

In [149]:
demeaned = arr - row_means.reshape((4, 1))

In [150]:
demeaned.mean(1)

array([-1.48029737e-16,  3.70074342e-17,  0.00000000e+00,  3.70074342e-17])

## Lan truyền trên các trục khác

Việc thực hiện một phép toán số học với một mảng có chiều thấp hơn trên các trục khác ngoài trục 0 là điều khá phổ biến. Theo quy tắc lan truyền (broadcasting), các "chiều lan truyền" trong mảng nhỏ hơn phải có giá trị là 1. Trong ví dụ về việc trừ trung bình hàng được trình bày ở đây, điều này có nghĩa là phải định hình lại mảng chứa trung bình các hàng thành hình dạng (4, 1) thay vì (4,):

In [151]:
arr - arr.mean(1).reshape((4, 1))

array([[ 0.01798636,  0.91140586, -0.92939222],
       [ 1.27521433, -0.51240718, -0.76280715],
       [-1.37269008,  0.58114876,  0.79154132],
       [-0.11546781, -0.68538247,  0.80085028]])

Trong trường hợp ba chiều, việc broadcast (lan truyền/phát sóng) trên bất kỳ chiều nào trong ba chiều chỉ là vấn đề định hình lại dữ liệu để có hình dạng tương thích

Do đó, một vấn đề phổ biến là cần phải thêm một trục mới với độ dài bằng 1 cụ thể cho mục đích broadcasting. Sử dụng hàm reshape là một lựa chọn, nhưng việc chèn một trục đòi hỏi phải xây dựng một tuple chỉ rõ hình dạng mới. Điều này thường có thể là một công việc tẻ nhạt. Vì vậy, các mảng NumPy cung cấp một cú pháp đặc biệt để chèn các trục mới thông qua việc đánh chỉ mục (indexing). Chúng ta sử dụng thuộc tính đặc biệt np.newaxis cùng với các lát cắt "đầy đủ" (full slices) để chèn trục mới:

In [152]:
arr = np.zeros((4, 4))

In [153]:
arr_3d = arr[:, np.newaxis, :]

In [154]:
arr_3d.shape

(4, 1, 4)

In [155]:
arr_1d = rng.standard_normal(3)

In [156]:
arr_1d[:, np.newaxis]

array([[ 0.31290292],
       [-0.13081169],
       [ 1.26998312]])

In [157]:
arr_1d[np.newaxis, :]

array([[ 0.31290292, -0.13081169,  1.26998312]])

Do đó, nếu chúng ta có một mảng ba chiều và muốn trừ đi giá trị trung bình (demean) của trục 2, chúng ta sẽ cần viết:

In [158]:
arr = rng.standard_normal((3, 4, 5))

In [159]:
depth_means = arr.mean(2)

In [160]:
depth_means

array([[ 0.04314136,  0.27468984, -0.18852342, -0.20137996],
       [-0.57324159, -0.54671393,  0.11832783, -0.63005577],
       [ 0.09723001,  0.59537117,  0.03307289, -0.6002202 ]])

In [161]:
depth_means.shape

(3, 4)

In [162]:
demeaned = arr - depth_means[:, :, np.newaxis]

In [163]:
demeaned.mean(2)

array([[ 4.44089210e-17, -1.11022302e-17,  8.88178420e-17,
        -1.66533454e-17],
       [ 2.22044605e-17, -4.44089210e-17, -2.22044605e-17,
        -4.44089210e-17],
       [ 4.44089210e-17,  6.66133815e-17,  0.00000000e+00,
         8.88178420e-17]])

Bạn có thể đang tự hỏi liệu có một cách nào để khái quát hóa việc trừ giá trị trung bình (demeaning) trên một trục mà không làm giảm hiệu suất hay không. Câu trả lời là có, nhưng nó đòi hỏi một số kỹ thuật đánh chỉ mục (indexing gymnastics):

In [164]:
def demean_axis(arr, axis=0):
    means = arr.mean(axis)
    # This generalizes things like [:, :, np.newaxis] to N dimensions
    indexer = [slice(None)] * arr.ndim
    indexer[axis] = np.newaxis
    return arr - means[indexer]

## Thiết lập Giá trị Mảng bằng Broadcasting

Quy tắc broadcasting chi phối các phép toán số học cũng áp dụng để thiết lập giá trị thông qua việc đánh chỉ mục mảng. Trong một trường hợp đơn giản, chúng ta có thể làm những việc như:

In [165]:
arr = np.zeros((4, 3))

In [166]:
arr[:] = 5

In [167]:
arr

array([[5., 5., 5.],
       [5., 5., 5.],
       [5., 5., 5.],
       [5., 5., 5.]])

Tuy nhiên, nếu chúng ta có một mảng một chiều chứa các giá trị mà chúng ta muốn gán vào các cột của mảng lớn hơn, chúng ta có thể làm điều đó miễn là hình dạng tương thích:

In [168]:
col = np.array([1.28, -0.42, 0.44, 1.6])

In [169]:
arr[:] = col[:, np.newaxis]

In [170]:
arr

array([[ 1.28,  1.28,  1.28],
       [-0.42, -0.42, -0.42],
       [ 0.44,  0.44,  0.44],
       [ 1.6 ,  1.6 ,  1.6 ]])

In [171]:
arr[:2] = [[-1.37], [0.509]]

In [172]:
arr

array([[-1.37 , -1.37 , -1.37 ],
       [ 0.509,  0.509,  0.509],
       [ 0.44 ,  0.44 ,  0.44 ],
       [ 1.6  ,  1.6  ,  1.6  ]])

# A.6. More About Sorting

Giống như kiểu list tích hợp sẵn của Python, phương thức sort của ndarray sẽ sắp xếp tại chỗ, nghĩa là nội dung của mảng sẽ được sắp xếp lại mà không tạo ra một mảng mới:

In [ ]:
rng = np.random.default_rng()

In [ ]:
arr = rng.standard_normal(6)

In [ ]:
arr.sort()

In [ ]:
arr

array([-0.87872261, -0.73847083, -0.69647456,  0.34883797,  0.41289716,
        1.35347772])

Khi sắp xếp mảng tại chỗ, hãy nhớ rằng nếu mảng đó là một view của một ndarray khác, thì mảng gốc cũng sẽ bị thay đổi:

In [ ]:
arr

array([-0.87872261, -0.73847083, -0.69647456,  0.34883797,  0.41289716,
        1.35347772])

Mặt khác, numpy.sort sẽ tạo ra một bản sao mới đã được sắp xếp của mảng. Ngoài ra, nó chấp nhận các tham số giống như phương thức sort của ndarray (ví dụ như kind):

In [ ]:
arr = rng.standard_normal(5)

In [ ]:
arr

array([ 0.39482212, -0.4303359 , -1.81212302, -0.2530782 , -0.38087369])

In [ ]:
np.sort(arr)

array([-1.81212302, -0.4303359 , -0.38087369, -0.2530782 ,  0.39482212])

In [ ]:
arr

array([ 0.39482212, -0.4303359 , -1.81212302, -0.2530782 , -0.38087369])

Tất cả các phương thức sắp xếp này đều nhận tham số axis để sắp xếp độc lập các phần dữ liệu dọc theo trục được truyền vào:

In [ ]:
arr = rng.standard_normal((3, 5))

In [ ]:
arr

array([[-1.41680289e+00,  7.85309956e-01,  7.08438872e-01,
         3.20518967e-02, -1.47516397e+00],
       [-3.01965154e-01, -9.96236650e-01,  1.14257540e+00,
        -1.08151576e-03,  5.17724262e-01],
       [ 1.43957692e-01, -7.14910782e-01,  1.96831965e-02,
         1.71500542e+00, -9.09857068e-01]])

In [ ]:
arr.sort(axis=1)

In [ ]:
arr

array([[-1.47516397e+00, -1.41680289e+00,  3.20518967e-02,
         7.08438872e-01,  7.85309956e-01],
       [-9.96236650e-01, -3.01965154e-01, -1.08151576e-03,
         5.17724262e-01,  1.14257540e+00],
       [-9.09857068e-01, -7.14910782e-01,  1.96831965e-02,
         1.43957692e-01,  1.71500542e+00]])

Nhận thấy rằng không có phương thức sắp xếp nào có tùy chọn để sắp xếp theo thứ tự giảm dần. Đây là một vấn đề trong thực tế vì việc cắt mảng sẽ tạo ra các view, do đó không tạo ra một bản sao hoặc không cần bất kỳ thao tác tính toán nào. Nhiều người dùng Python quen với “mẹo” rằng với một danh sách các giá trị, values[::-1] sẽ trả về danh sách theo thứ tự ngược lại. Điều này cũng đúng với ndarray:



In [ ]:
arr[:, ::-1]

array([[ 7.85309956e-01,  7.08438872e-01,  3.20518967e-02,
        -1.41680289e+00, -1.47516397e+00],
       [ 1.14257540e+00,  5.17724262e-01, -1.08151576e-03,
        -3.01965154e-01, -9.96236650e-01],
       [ 1.71500542e+00,  1.43957692e-01,  1.96831965e-02,
        -7.14910782e-01, -9.09857068e-01]])

## Sắp xếp gián tiếp: argsort và lexsort

Trong phân tích dữ liệu, bạn có thể cần sắp xếp lại các bộ dữ liệu theo một hoặc nhiều khóa. Ví dụ, một bảng dữ liệu về các sinh viên có thể cần được sắp xếp theo họ, sau đó là tên. Đây là một ví dụ về sắp xếp gián tiếp, và nếu bạn đã đọc các chương liên quan đến pandas, bạn đã thấy nhiều ví dụ cấp cao hơn. Khi có một hoặc nhiều khóa (một mảng giá trị hoặc nhiều mảng giá trị), bạn muốn nhận được một mảng các chỉ số nguyên (thường gọi là indexers) cho biết cách sắp xếp lại dữ liệu để có thứ tự mong muốn. Hai phương pháp cho việc này là argsort và numpy.lexsort. Ví dụ:

In [ ]:
values = np.array([5, 0, 1, 3, 2])

In [ ]:
indexer = values.argsort()

In [ ]:
indexer

array([1, 2, 4, 3, 0])

In [ ]:
values[indexer]

array([0, 1, 2, 3, 5])

Là một ví dụ phức tạp hơn, đoạn mã này sẽ sắp xếp lại một mảng hai chiều dựa trên hàng đầu tiên của nó:

In [ ]:
arr = rng.standard_normal((3, 5))

In [ ]:
arr[0] = values

In [ ]:
arr

array([[ 5.        ,  0.        ,  1.        ,  3.        ,  2.        ],
       [ 0.90731116, -1.20703181, -1.70631191,  1.13994985, -0.70625983],
       [-1.39534688,  0.60677022, -0.30496205,  0.06618859,  0.8367133 ]])

In [ ]:
arr[:, arr[0].argsort()]

array([[ 0.        ,  1.        ,  2.        ,  3.        ,  5.        ],
       [-1.20703181, -1.70631191, -0.70625983,  1.13994985,  0.90731116],
       [ 0.60677022, -0.30496205,  0.8367133 ,  0.06618859, -1.39534688]])

lexsort tương tự như argsort, nhưng nó thực hiện sắp xếp gián tiếp theo thứ tự từ điển trên nhiều mảng khóa. Giả sử chúng ta muốn sắp xếp một số dữ liệu được xác định bởi tên và họ:

In [ ]:
first_name = np.array(['Bob', 'Jane', 'Steve', 'Bill', 'Barbara'])

In [ ]:
last_name = np.array(['Jones', 'Arnold', 'Arnold', 'Jones', 'Walters'])

In [ ]:
sorter = np.lexsort((first_name, last_name))

In [ ]:
sorter

array([1, 2, 3, 0, 4])

In [ ]:
list(zip(last_name[sorter], first_name[sorter]))

[(np.str_('Arnold'), np.str_('Jane')),
 (np.str_('Arnold'), np.str_('Steve')),
 (np.str_('Jones'), np.str_('Bill')),
 (np.str_('Jones'), np.str_('Bob')),
 (np.str_('Walters'), np.str_('Barbara'))]

lexsort có thể gây nhầm lẫn khi bạn sử dụng lần đầu, vì thứ tự các khóa được dùng để sắp xếp dữ liệu bắt đầu từ mảng cuối cùng được truyền vào. Trong ví dụ này, last_name được sử dụng trước first_name.

## Các thuật toán sắp xếp thay thế



Một thuật toán sắp xếp ổn định sẽ giữ nguyên vị trí tương đối của các phần tử bằng nhau. Điều này đặc biệt quan trọng trong các phép sắp xếp gián tiếp, nơi thứ tự tương đối có ý nghĩa:

In [ ]:
values = np.array(['2:first', '2:second', '1:first', '1:second', '1:third'])

In [ ]:
key = np.array([2, 2, 1, 1, 1])

In [ ]:
indexer = key.argsort(kind='mergesort')

In [ ]:
indexer

array([2, 3, 4, 0, 1])

In [ ]:
values.take(indexer)

array(['1:first', '1:second', '1:third', '2:first', '2:second'],
      dtype='<U8')

Thuật toán sắp xếp ổn định duy nhất có sẵn là mergesort, với hiệu suất đảm bảo O(n log n), nhưng hiệu suất trung bình của nó thường kém hơn so với phương pháp quicksort mặc định. Xem Bảng A-3 để biết tóm tắt các phương pháp có sẵn và hiệu suất tương đối (cũng như các đảm bảo về hiệu suất). Đây không phải là điều mà hầu hết người dùng cần quan tâm, nhưng biết về nó cũng hữu ích.

## Sắp xếp một phần mảng

Một trong những mục tiêu của việc sắp xếp có thể là xác định các phần tử lớn nhất hoặc nhỏ nhất trong một mảng. NumPy có các phương thức nhanh, numpy.partition và np.argpartition, để phân vùng một mảng xung quanh phần tử nhỏ nhất thứ k:

In [ ]:
rng = np.random.default_rng(12345)

In [ ]:
arr = rng.standard_normal(20)

In [ ]:
arr

array([-1.42382504,  1.26372846, -0.87066174, -0.25917323, -0.07534331,
       -0.74088465, -1.3677927 ,  0.6488928 ,  0.36105811, -1.95286306,
        2.34740965,  0.96849691, -0.75938718,  0.90219827, -0.46695317,
       -0.06068952,  0.78884434, -1.25666813,  0.57585751,  1.39897899])

In [ ]:
np.partition(arr, 3)

array([-1.95286306, -1.42382504, -1.3677927 , -1.25666813, -0.87066174,
       -0.75938718, -0.74088465, -0.46695317, -0.25917323, -0.07534331,
       -0.06068952,  0.36105811,  0.57585751,  0.6488928 ,  0.78884434,
        0.90219827,  0.96849691,  1.26372846,  1.39897899,  2.34740965])

In [ ]:
indices = np.argpartition(arr, 3)

In [ ]:
indices

array([ 9,  0,  6, 17,  2, 12,  5, 14,  3,  4, 15,  8, 18,  7, 16, 13, 11,
        1, 19, 10])

In [ ]:
arr.take(indices)

array([-1.95286306, -1.42382504, -1.3677927 , -1.25666813, -0.87066174,
       -0.75938718, -0.74088465, -0.46695317, -0.25917323, -0.07534331,
       -0.06068952,  0.36105811,  0.57585751,  0.6488928 ,  0.78884434,
        0.90219827,  0.96849691,  1.26372846,  1.39897899,  2.34740965])

numpy.searchsorted: Tìm phần tử trong một mảng đã được sắp xếp

In [ ]:
arr = np.array([0, 1, 7, 12, 15])

In [ ]:
arr.searchsorted(9)

np.int64(3)

Bạn cũng có thể truyền vào một mảng giá trị để nhận lại một mảng các chỉ số:

In [ ]:
arr.searchsorted([0, 8, 11, 16])

array([0, 3, 3, 5])

Bạn có thể đã nhận thấy rằng searchsorted trả về 0 cho phần tử 0. Điều này là do hành vi mặc định là trả về chỉ số ở phía bên trái của một nhóm các giá trị bằng nhau:

In [ ]:
arr = np.array([0, 0, 0, 1, 1, 1, 1])

In [ ]:
arr.searchsorted([0, 1])

array([0, 3])

In [ ]:
arr.searchsorted([0, 1], side='right')

array([3, 7])

Là một ứng dụng khác của searchsorted, giả sử chúng ta có một mảng các giá trị nằm trong khoảng từ 0 đến 10.000, và một mảng riêng các “cạnh của các thùng” mà chúng ta muốn dùng để phân nhóm dữ liệu:

In [ ]:
data = np.floor(rng.uniform(0, 10000, size=50))

In [ ]:
bins = np.array([0, 100, 1000, 5000, 10000])

In [ ]:
data

array([ 815., 1598., 3401., 4651., 2664., 8157., 1932., 1294.,  916.,
       5985., 8547., 6016., 9319., 7247., 8605., 9293., 5461., 9376.,
       4949., 2737., 4517., 6650., 3308., 9034., 2570., 3398., 2588.,
       3554.,   50., 6286., 2823.,  680., 6168., 1763., 3043., 4408.,
       1502., 2179., 4743., 4763., 2552., 2975., 2790., 2605., 4827.,
       2119., 4956., 2462., 8384., 1801.])

Để gán nhãn cho mỗi điểm dữ liệu thuộc về khoảng nào (trong đó 1 nghĩa là thuộc thùng 0, 100)), chúng ta chỉ cần sử dụng searchsorted:

In [ ]:
labels = bins.searchsorted(data)

In [ ]:
labels

array([2, 3, 3, 3, 3, 4, 3, 3, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 4,
       3, 4, 3, 3, 3, 3, 1, 4, 3, 2, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 4, 3])

Điều này, kết hợp với groupby của pandas, có thể được sử dụng để phân nhóm dữ liệu theo các thùng (bin):

In [ ]:
pd.Series(data).groupby(labels).mean()

1      50.000000
2     803.666667
3    3079.741935
4    7635.200000
dtype: float64

# A.5. Structured and Record Arrays

Bạn có thể đã nhận thấy cho đến bây giờ rằng ndarray là một kiểu dữ liệu đồng nhất; nghĩa là, nó biểu diễn một khối bộ nhớ mà mỗi phần tử chiếm cùng một số byte, được xác định bởi kiểu dữ liệu. Nhìn bề ngoài, điều này có vẻ như không cho phép bạn biểu diễn dữ liệu không đồng nhất hoặc dạng bảng. Một mảng có cấu trúc (structured array) là một ndarray mà mỗi phần tử có thể được xem như là một struct trong C (do đó có tên “structured”) hoặc một dòng trong bảng SQL với nhiều trường có tên khác nhau:

In [ ]:
# import thư viện
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
dtype = [('x', np.float64), ('y', np.int32)]

In [ ]:
sarr = np.array([(1.5, 6), (np.pi, -2)], dtype=dtype)

In [ ]:
sarr

array([(1.5       ,  6), (3.14159265, -2)],
      dtype=[('x', '<f8'), ('y', '<i4')])

Có nhiều cách để xác định kiểu dữ liệu có cấu trúc (xem tài liệu trực tuyến của NumPy). Một cách phổ biến là sử dụng danh sách các tuple với dạng (tên_trường, kiểu_dữ_liệu_trường). Bây giờ, các phần tử của mảng là các đối tượng giống tuple, các phần tử của chúng có thể được truy cập như một từ điển:

In [ ]:
sarr[0]

np.void((1.5, 6), dtype=[('x', '<f8'), ('y', '<i4')])

In [ ]:
sarr[0]['y']

np.int32(6)

Tên các trường được lưu trong thuộc tính dtype.names. Khi bạn truy cập một trường trên mảng có cấu trúc, bạn sẽ nhận được một view dạng strided trên dữ liệu, do đó không có dữ liệu nào bị sao chép:

In [ ]:
sarr['x']

array([1.5       , 3.14159265])

## Kiểu dữ liệu lồng nhau và các trường đa chiều



Khi xác định kiểu dữ liệu có cấu trúc, bạn cũng có thể truyền thêm một shape (dưới dạng số nguyên hoặc tuple):

In [ ]:
dtype = [('x', np.int64, 3), ('y', np.int32)]

In [ ]:
arr = np.zeros(4, dtype=dtype)

In [ ]:
arr

array([([0, 0, 0], 0), ([0, 0, 0], 0), ([0, 0, 0], 0), ([0, 0, 0], 0)],
      dtype=[('x', '<i8', (3,)), ('y', '<i4')])

Trong trường hợp này, trường x bây giờ sẽ là một mảng có độ dài 3 cho mỗi bản ghi:

In [ ]:
arr[0]['x']

array([0, 0, 0])

Thuận tiện là, khi truy cập arr['x'] thì sẽ trả về một mảng hai chiều thay vì một mảng một chiều như các ví dụ trước:

In [ ]:
arr['x']

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

Điều này cho phép bạn biểu diễn các cấu trúc lồng nhau phức tạp hơn dưới dạng một khối bộ nhớ duy nhất trong một mảng. Bạn cũng có thể lồng các kiểu dữ liệu để tạo ra các cấu trúc phức tạp hơn. Dưới đây là một ví dụ:

In [ ]:
dtype = [('x', [('a', 'f8'), ('b', 'f4')]), ('y', np.int32)]

In [ ]:
data = np.array([((1, 2), 5), ((3, 4), 6)], dtype=dtype)

In [ ]:
data['x']

array([(1., 2.), (3., 4.)], dtype=[('a', '<f8'), ('b', '<f4')])

In [ ]:
data['y']

array([5, 6], dtype=int32)

In [ ]:
data['x']['a']

array([1., 3.])

pandas DataFrame không hỗ trợ tính năng này theo cách tương tự, mặc dù nó khá giống với chỉ mục phân cấp (hierarchical indexing).

## A.7 Viết hàm NumPy nhanh với Numba

### Giới thiệu Numba và @jit


In [ ]:
import numpy as np

def mean_distance(x, y):
    nx = len(x)
    result = 0.0
    count = 0
    for i in range(nx):
        result += x[i] - y[i]
        count += 1
    return result / count

In [ ]:
x = rng.standard_normal(10_000_000)
y = rng.standard_normal(10_000_000)
%timeit mean_distance(x, y)

2.2 s ± 393 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit (x - y).mean()

43.5 ms ± 3.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
import numba as nb

numba_mean_distance = nb.jit(mean_distance)

In [ ]:
@nb.jit
def numba_mean_distance(x, y):
    nx = len(x)
    result = 0.0
    count = 0
    for i in range(nx):
        result += x[i] - y[i]
        count += 1
    return result / count

In [ ]:
%timeit numba_mean_distance(x, y)

7.25 ms ± 1.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
from numba import float64, njit
@njit(float64(float64[:], float64[:]))
def mean_distance(x, y):
    return (x - y).mean()

### Tạo ufunc tùy chỉnh với @vectorize


In [ ]:
from numba import vectorize
@vectorize
def nb_add(x, y):
    return x + y

In [ ]:
x = np.arange(10)
nb_add(x, x)

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

In [ ]:
nb_add.accumulate(x, 0)

array([ 0,  1,  3,  6, 10, 15, 21, 28, 36, 45])

## A.9 Mẹo về Hiệu suất


### Tầm quan trọng của Bộ nhớ Liền kề (Contiguous Memory)


In [ ]:
arr_c = np.ones((100, 10000), order='C')
arr_f = np.ones((100, 10000), order='F')
arr_c.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [ ]:
arr_f.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [ ]:
arr_f.flags.f_contiguous

True

In [ ]:
%timeit arr_c.sum(1)

398 μs ± 53.9 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%timeit arr_f.sum(1)

256 μs ± 53.1 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
arr_f.copy('C').flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [ ]:
arr_c[:50].flags.contiguous

True

In [ ]:
arr_c[:, :50].flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False